#1. Raw


In [1]:
import os
import shutil

# Reiniciamos el entorno para asegurar limpieza
if os.path.exists('datalake'):
    shutil.rmtree('datalake')

# Creamos las tres zonas de modelado
for layer in ['datalake/bronze', 'datalake/silver', 'datalake/gold']:
    os.makedirs(layer, exist_ok=True)

print("✅ Infraestructura de Data Lake creada.")

✅ Infraestructura de Data Lake creada.


#2. Bronze

In [2]:
import requests
import json

base_url = "https://fakestoreapi.com"
endpoints = ['products', 'users', 'carts']

print("📥 Ingestando datos crudos...")

for e in endpoints:
    # Simulamos la extracción de la fuente
    raw_data = requests.get(f"{base_url}/{e}").json()

    # Persistencia en formato nativo (JSON)
    with open(f'datalake/bronze/{e}.json', 'w') as f:
        json.dump(raw_data, f)

print(f"   -> {len(endpoints)} entidades cargadas en Bronze.")

📥 Ingestando datos crudos...
   -> 3 entidades cargadas en Bronze.


#3. Silver

In [3]:
import pandas as pd

print("⚙️ Aplicando reglas de calidad (Silver Layer)...")

# --- 1. PRODUCTOS ---
# Regla: El precio viene como string o int, forzamos a Float para consistencia financiera.
df_prod = pd.read_json('datalake/bronze/products.json')
df_prod['price'] = df_prod['price'].astype(float)
df_prod.to_parquet('datalake/silver/products.parquet')

# --- 2. USUARIOS ---
# Regla de Modelado: Aplanar la jerarquía.
# 'address' es un diccionario. Lo convertimos en columnas: address_city, address_street...
with open('datalake/bronze/users.json') as f:
    users_data = json.load(f)

# json_normalize es nuestra herramienta de normalización
df_users = pd.json_normalize(users_data, sep='_')
df_users.to_parquet('datalake/silver/users.parquet')

# --- 3. CARRITOS (Transacciones) ---
# Regla: Las fechas deben ser objetos manipulables, no texto.
df_carts = pd.read_json('datalake/bronze/carts.json')
df_carts['date'] = pd.to_datetime(df_carts['date'])
df_carts.to_parquet('datalake/silver/carts.parquet')

print("✅ Datos limpios y estandarizados en formato Parquet.")

⚙️ Aplicando reglas de calidad (Silver Layer)...
✅ Datos limpios y estandarizados en formato Parquet.


#4.1 Gold - Dimensiones

In [4]:
# Leemos desde Silver (nuestra fuente limpia)
s_prod = pd.read_parquet('datalake/silver/products.parquet')
s_user = pd.read_parquet('datalake/silver/users.parquet')

# --- DIMENSIÓN PRODUCTO ---
# Selección de atributos maestros
dim_product = s_prod[['id', 'title', 'category', 'price']].copy()
dim_product.columns = ['product_id', 'product_name', 'category', 'unit_price']

# --- DIMENSIÓN USUARIO ---
# Selección de atributos demográficos y geográficos
dim_user = s_user[['id', 'username', 'email', 'address_city']].copy()
dim_user.columns = ['user_id', 'username', 'email', 'city']

# Materialización
dim_product.to_parquet('datalake/gold/dim_product.parquet')
dim_user.to_parquet('datalake/gold/dim_user.parquet')

print("✅ Dimensiones creadas.")

✅ Dimensiones creadas.


In [5]:
s_user.head()

,id,email,username,password,phone,__v,address_geolocation_lat,address_geolocation_long,address_city,address_street,address_number,address_zipcode,name_firstname,name_lastname
0,1,john@gmail.com,johnd,m38rmF$,1-570-236-7033,0,-37.3159,81.1496,kilcoole,new road,7682,12926-3874,john,doe
1,2,morrison@gmail.com,mor_2314,83r5^_,1-570-236-7033,0,-37.3159,81.1496,kilcoole,Lovers Ln,7267,12926-3874,david,morrison
2,3,kevin@gmail.com,kevinryan,kev02937@,1-567-094-1345,0,40.3467,-30.1310,Cullman,Frances Ct,86,29567-1452,kevin,ryan
3,4,don@gmail.com,donero,ewedon,1-765-789-6734,0,50.3467,-20.1310,San Antonio,Hunters Creek Dr,6454,98234-1734,don,romer
4,5,derek@gmail.com,derek,jklg*_56,1-956-001-1945,0,40.3467,-40.1310,san Antonio,adams St,245,80796-1234,derek,powell


#4.2 Gold - Hecho

In [6]:
s_cart = pd.read_parquet('datalake/silver/carts.parquet')

# 1. CAMBIO DE GRANULARIDAD (La clave del modelado)
# Convertimos la lista de productos en filas individuales
fact_sales = s_cart.explode('products').reset_index(drop=True)

# 2. EXTRACCIÓN DE LLAVES FORÁNEAS (FK)
# Ahora cada fila tiene un diccionario {'productId': 1, 'quantity': 2}
# Extraemos esos valores para que sean columnas relacionables
fact_sales['product_id'] = fact_sales['products'].apply(lambda x: x['productId'])
fact_sales['quantity'] = fact_sales['products'].apply(lambda x: x['quantity'])

# 3. ENRIQUECIMIENTO (Cálculo de Métricas)
# El hecho "Venta" necesita el monto ($). El carrito no trae precios.
# Hacemos un JOIN con la dimensión producto para traer el precio unitario.
fact_sales = fact_sales.merge(dim_product[['product_id', 'unit_price']], on='product_id', how='left')

# Métrica Calculada: Cantidad * Precio Unitario
fact_sales['total_amount'] = fact_sales['quantity'] * fact_sales['unit_price']

# 4. DIMENSIÓN TIEMPO DERIVADA
# Extraemos atributos de fecha para facilitar el filtrado
fact_sales['year'] = fact_sales['date'].dt.year
fact_sales['month'] = fact_sales['date'].dt.month
fact_sales['day'] = fact_sales['date'].dt.day

# 5. SELECCIÓN FINAL (Esquema Estrella)
# Nos quedamos solo con las llaves (FK) y las métricas numéricas
final_columns = [
    'id',           # Sale ID (Podemos usar el Cart ID como referencia)
    'userId',       # FK hacia Dim_User
    'product_id',   # FK hacia Dim_Product
    'date', 'year', 'month', # Dimensiones Temporales
    'quantity',     # Métrica 1
    'total_amount'  # Métrica 2
]

fact_sales_final = fact_sales[final_columns].rename(columns={'id': 'sale_id', 'userId': 'user_id'})

# Materialización
fact_sales_final.to_parquet('datalake/gold/fact_sales.parquet')

print("✅ Tabla de Hechos (Fact_Sales) creada con granularidad de línea de producto.")

✅ Tabla de Hechos (Fact_Sales) creada con granularidad de línea de producto.


In [7]:
fact_sales_final.head()

,sale_id,user_id,product_id,date,year,month,quantity,total_amount
0,1,1,1,2020-03-02 00:00:00+00:00,2020,3,4,439.80
1,1,1,2,2020-03-02 00:00:00+00:00,2020,3,1,22.30
2,1,1,3,2020-03-02 00:00:00+00:00,2020,3,6,335.94
3,2,1,2,2020-01-02 00:00:00+00:00,2020,1,4,89.20
4,2,1,1,2020-01-02 00:00:00+00:00,2020,1,10,1099.50


#5. Validacion

In [8]:
# Cargamos el modelo (simulando Power BI o Tableau)
gold_fact = pd.read_parquet('datalake/gold/fact_sales.parquet')
gold_prod = pd.read_parquet('datalake/gold/dim_product.parquet')

# Hacemos el JOIN entre Hechos y Dimensiones
modelo_estrella = gold_fact.merge(gold_prod, on='product_id')

# Agregación (GROUP BY)
reporte = modelo_estrella.groupby('category')[['quantity', 'total_amount']].sum().sort_values('total_amount', ascending=False)

# Formato visual
pd.options.display.float_format = '${:,.2f}'.format
print(reporte)

                  quantity  total_amount
category                                
men's clothing          31     $2,646.44
jewelery                 4     $1,410.98
electronics              6       $624.00
women's clothing         1         $9.85


#6. Preguntas de Negocio a Resolver

In [9]:
#6.1 ¿Cuál es el Ingreso Total Histórico (Total Revenue) de la compañía?
total_revenue = gold_fact['total_amount'].sum()
print(f"💰 Ingreso Total: ${total_revenue:,.2f}")

💰 Ingreso Total: $4,691.27


In [10]:
#6.2 ¿Cuántas unidades (Quantity) se han vendido en total?
total_units = gold_fact['quantity'].sum()
print(f"📦 Unidades Vendidas: {total_units:,}")

📦 Unidades Vendidas: 42


In [11]:
#6.3 ¿Cuál es el promedio de precio de venta (Average Selling Price)?
avg_price = gold_prod['unit_price'].mean()
print(f"💵 Precio Promedio: ${avg_price:,.2f}")

💵 Precio Promedio: $162.05


In [12]:
#6.4 ¿Cuál es la Categoría de Productos más rentable?
categoria_rentable = modelo_estrella.groupby('category')['total_amount'].sum().sort_values(ascending=False)
print("📊 Ingresos por Categoría:")
print(categoria_rentable)

📊 Ingresos por Categoría:
category
men's clothing     $2,646.44
jewelery           $1,410.98
electronics          $624.00
women's clothing       $9.85
Name: total_amount, dtype: float64


In [13]:
#6.5 Top 5: ¿Cuáles son los productos "Estrella" (Más vendidos por ingresos)?
top_productos = modelo_estrella.groupby('product_name')['total_amount'].sum().sort_values(ascending=False).head(5)
print("⭐ Top 5 Productos Estrella:")
print(top_productos)

⭐ Top 5 Productos Estrella:
product_name
Fjallraven - Foldsack No. 1 Backpack, Fits 15 Laptops                         $2,199.00
John Hardy Women's Legends Naga Gold & Silver Dragon Station Chain Bracelet   $1,390.00
WD 4TB Gaming Drive Works with Playstation 4 Portable External Hard Drive       $342.00
Mens Cotton Jacket                                                              $335.94
SanDisk SSD PLUS 1TB Internal SSD - SATA III 6 Gb/s                             $218.00
Name: total_amount, dtype: float64


In [14]:
#6.6 ¿Cuál es la ciudad (City) con mayor volumen de compras?
gold_user = pd.read_parquet('datalake/gold/dim_user.parquet')
ventas_por_ciudad = gold_fact.merge(gold_user, on='user_id').groupby('city')['total_amount'].sum().sort_values(ascending=False)
print("🌆 Ventas por Ciudad:")
print(ventas_por_ciudad.head(10))

🌆 Ventas por Ciudad:
city
kilcoole      $3,660.64
San Antonio     $560.00
Cullman         $460.78
mesa              $9.85
Name: total_amount, dtype: float64


#7. Análisis de Tendencia

In [16]:
#¿Cómo se comportan las ventas por mes?
ventas_mensuales = gold_fact.groupby('month')['total_amount'].sum().sort_index()
print("📈 Tendencia de Ventas Mensuales:")
print(ventas_mensuales)

# Opcional: Graficar
#import matplotlib.pyplot as plt
#ventas_mensuales.plot(kind='line', marker='o')
#plt.title('Ventas por Mes')
#plt.xlabel('Mes')
#plt.ylabel('Ingresos ($)')
#plt.show()

📈 Tendencia de Ventas Mensuales:
month
1   $3,018.50
3   $1,672.77
Name: total_amount, dtype: float64


#8. ¿Cuál es el "Ticket Promedio" (AOV - Average Order Value)?

In [17]:
ticket_por_venta = gold_fact.groupby('sale_id')['total_amount'].sum()
aov = ticket_por_venta.mean()
print(f"🛒 Ticket Promedio (AOV): ${aov:,.2f}")

🛒 Ticket Promedio (AOV): $670.18


#9. ¿Quiénes son los clientes "VIP"? (Top 3 clientes por gasto acumulado)

In [18]:
clientes_vip = gold_fact.merge(gold_user, on='user_id').groupby('username')['total_amount'].sum().sort_values(ascending=False).head(3)
print("👑 Top 3 Clientes VIP:")
print(clientes_vip)

👑 Top 3 Clientes VIP:
username
johnd       $3,376.74
donero        $560.00
kevinryan     $460.78
Name: total_amount, dtype: float64


#Reto de Validación Final

In [19]:
# Reto de Validación
total_ventas = modelo_estrella['total_amount'].sum()
ventas_electronics = modelo_estrella[modelo_estrella['category'] == 'electronics']['total_amount'].sum()
porcentaje = (ventas_electronics / total_ventas) * 100

print(f"🔌 Participación de Electronics: {porcentaje:.2f}%")

🔌 Participación de Electronics: 13.30%
